# This code will calculate Entrapment conditions given host and inclusions.
## To run calculate do the following:
    1. Set the host and inclusions.
    2. Specify the mole fractions of hosts and inclusions.
    3. Specify the Inclusion Pressure
    4. To calculate the Entrapment Pressure, specify the Entrapment Temperature.
    5. To calculate the Entrapment Temperature, specify the Entrapment Pressure.
    6. To make a plot, specify the Entrapment Pressure. 

In [1]:
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import inclusion_calc as ic
import matplotlib.pyplot as plt
import numpy as np


calc = ic.BarInclusion()
all_options = ['--'] + calc.df['End-member'].values.tolist()


mapping = {'First': 1, 'Second': 2, 'Third': 3, 'Fourth': 4, 'Fifth': 5}
host_widgets = {}
inclusion_widgets = {}
solution_state = {'host': {'Almandine': 1.0}, 'inclusion': {'Albite': 1.0}}

def member_value_change(change):
    if 'host' in change['owner'].description.lower():
        if change['new'] not in solution_state['host'].keys():
            key = mapping[change['owner'].description.split('Host')[0]]
            solution_state['host'][change['new']] = host_widgets[key]['mole'].value
            solution_state['host'].pop(change['old'], None)
    else:
        if change['new'] not in solution_state['inclusion'].keys():
            key = mapping[change['owner'].description.split('Inclusion')[0]]
            solution_state['inclusion'][change['new']] = inclusion_widgets[key]['mole'].value
            solution_state['inclusion'].pop(change['old'], None)
    
def mole_value_change(change):
    if 'host' in change['owner'].description.lower():
        key = mapping[change['owner'].description.split('Host')[0]]
        if host_widgets[key]['end-member'].value in solution_state['host'].keys():
            solution_state['host'][host_widgets[key]['end-member'].value] = change['new']
    else:
        key = mapping[change['owner'].description.split('Inclusion')[0]]
        if inclusion_widgets[key]['end-member'].value in solution_state['inclusion'].keys():
            solution_state['inclusion'][inclusion_widgets[key]['end-member'].value] = change['new']

style = {'description_width': 'initial'}

for key, value in mapping.items():
    host_name = key + 'Host :'
    host_mole_name = key + 'Host, mole fraction:'
    if value == 1:
        host_widgets[value] = {
        'end-member': widgets.Dropdown(
            options=all_options,
            value='Almandine',
            description=host_name,
            disabled=False,
            style=style),
        
        'mole': widgets.BoundedFloatText(
            value=1.0,
            min=0,
            max=1.0,
            step=0.01,
            description=host_mole_name,
            disabled=False,
            style=style)
        }
    else:
        host_widgets[value] = {
            'end-member': widgets.Dropdown(
                options=all_options,
                value='--',
                description=host_name,
                disabled=False,
                style=style),

            'mole': widgets.BoundedFloatText(
                value=0.0,
                min=0,
                max=1.0,
                step=0.01,
                description=host_mole_name,
                disabled=False,
                style=style)
        }
    
    inclusion_name = key + 'Inclusion :'
    inclusion_mole_name = key + 'Inclusion, mole fraction:'
    if value == 1:
         inclusion_widgets[value] = {
        'end-member': widgets.Dropdown(
            options=all_options,
            value='Albite',
            description=inclusion_name,
            disabled=False,
            style=style),
        'mole': widgets.BoundedFloatText(
            value=1.0,
            min=0,
            max=1.0,
            step=0.01,
            description=inclusion_mole_name,
            disabled=False,
            style=style)
         }
    else:
        inclusion_widgets[value] = {
            'end-member': widgets.Dropdown(
                options=all_options,
                value='--',
                description=inclusion_name,
                disabled=False,
                style=style),
            'mole': widgets.BoundedFloatText(
                value=0.0,
                min=0,
                max=1.0,
                step=0.01,
                description=inclusion_mole_name,
                disabled=False,
                style=style)
        }
    host_widgets[value]['end-member'].observe(member_value_change, names='value')
    host_widgets[value]['mole'].observe(mole_value_change, names='value')
    inclusion_widgets[value]['end-member'].observe(member_value_change, names='value')
    inclusion_widgets[value]['mole'].observe(mole_value_change, names='value')



widgets.VBox([widgets.HBox([x['end-member'], x['mole']]) for x in host_widgets.values()] 
             + [widgets.HBox([x['end-member'], x['mole']]) for x in inclusion_widgets.values()] )



In [2]:
P_incl = widgets.FloatText(
            value=100.0,
            description='Inclusion Pressure, MPa',
            disabled=False,
            style=style)
P_entrap = widgets.FloatText(
            value=100.0,
            description='Entrapment Pressure, MPa',
            disabled=False,
            style=style)
T_entrap = widgets.FloatText(
            value=300.0,
            description='Entrapment Temperature, C',
            disabled=False,
            style=style)
calc_P_entrap = widgets.Button(
    description='Calculate Entrapment Pressure',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='calculator',
    layout=widgets.Layout(width='auto'),
)
calc_T_entrap = widgets.Button(
    description='Calculate Entrapment Temperature',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='calculator',
    layout=widgets.Layout(width='auto'),
)
def click_p_entrap(b):
    try:
        i.clear_output()
        output = calc.P_entrap_calc(P_incl.value / 100.0, 
                                    T_entrap.value + 273.15, 
                                    solution_state['inclusion'], 
                                    solution_state['host'])
        i.append_stdout('Entrapment pressure is : {0:5.3f} MPa'.format(output))
    except:
        pass
    
def click_t_entrap(b):
    try:
        i.clear_output()
        output = calc.T_entrap_calc(P_incl.value / 100.0, 
                                    P_entrap.value / 100.0, 
                                    solution_state['inclusion'], 
                                    solution_state['host'])
        i.append_stdout('Entrapment temperature is : {0:5.3f} C'.format(output - 273.15))
    except:
        pass
calc_P_entrap.on_click(click_p_entrap)
widgets.VBox([widgets.VBox([P_incl, P_entrap, T_entrap]),
             widgets.HBox([calc_P_entrap, calc_T_entrap])])
calc_T_entrap.on_click(click_t_entrap)
widgets.VBox([widgets.VBox([P_incl, P_entrap, T_entrap]),
             widgets.HBox([calc_P_entrap, calc_T_entrap])])

In [3]:
i = widgets.Output()
display(i)

Output()

In [4]:
P_range = np.linspace(1, 4, 3)  # Range of pressures to calculate in kbar
T_range = np.linspace(400, 1000, 6) + 273.15  # Range of temperatures to calculate in K


make_plot = widgets.Button(
    description='Make isobar plot',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='calculator',
    layout=widgets.Layout(width='auto'),
)
def click_plot(b):
    clear_output()
    display(make_plot)
    P_entrap, T_mat, P_mat = calc.isobar_calc(T_range, 
                                              P_range, 
                                              solution_state['inclusion'], 
                                              solution_state['host'])
    ax = plt.plot(T_mat.T - 273.15, 100*P_entrap.T, 's-', linewidth=3)       
    plt.xlabel('Temperature, $^\circ$C')
    plt.ylabel('Entrapment pressure, MPa')
    plt.grid(True)
#     display(ax.show())

make_plot.on_click(click_plot)


In [5]:
# j = widgets.Output()
# display(j)
display(make_plot)

Button(button_style='success', description='Make isobar plot', icon='calculator', layout=Layout(width='auto'),…

In [6]:
%%HTML
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>